Using this data:

- https://re.jrc.ec.europa.eu/pvg_tools/en/tools.html
- https://joint-research-centre.ec.europa.eu/photovoltaic-geographical-information-system-pvgis/getting-started-pvgis/api-non-interactive-service_en#ref-4-python-wrap-for-api-calls

Info
- Entry point: PVGIS 5.3: https://re.jrc.ec.europa.eu/api/v5_3/tool_name?param1=value1&param2=value2&...
- variables:
- tool_name: PVcalc, SHScalc, MRcalc, DRcalc, seriescalc, tmy, printhorizon. 
ours is HRcalc?? HR?

- param1=value1, param2=value2, ...: input parameters of the tool with their corresponding values concatenated in a query string format.

- API functions accept GET method only
- API calls have a rate limit of 30 calls/second per IP address.

- outputformat=json JSON output.

Our plan:
- take radiation data from this site to feed into our model / similar model to ours
- process radiation data available from site,
- feed to model, 
- predict PV production,
- return prediction



### LatLon input function

Unsure whether we'll need it but keeping here for later use.

In [ ]:
# copy pasted from API page. figure out. regarding the specific format of the latlons

def format_latlon(x,y):
    lon = float(point_WGS84.split(' ')[1][:-1])
    lat = float(point_WGS84.split(' ')[0][:-1])
    return (lat, lon)

In [2]:
import sys
import urllib.parse
import requests
import os, csv, json, requests
import glob
import pandas as pd
import numpy as np
from datetime import date

BASE_URI = 'https://re.jrc.ec.europa.eu/api/seriescalc' # for the hourly radiation

lat = 52.520008 # Berlin coords
lon = 13.404954
startyear = 2020
endyear = 2021

params = {
    "lat": lat,
    "lon": lon,
    "startyear": startyear,
    "endyear": endyear,
    "pvcalculation": 1, # If "0" outputs only solar radiation calculations,
        # if "1" outputs the estimation of hourly PV production as well.
    "peakpower": 86.4, # need if inputting pvcalc as 1
        # our microgrid 'rated output power' is 86.4kWp
    "loss": 0,
    # "components": , # If "1" outputs beam, diffuse and reflected radiation components.
        #Otherwise, it outputs only global values.
    "outputformat": "json",
    # "browser": # unsure if needed
}

# don't know if we need the below. try without first
# params = "&".join([f'{key}={value}' for key, value in pvgis_params.items()])
# url_seriescalc = f'{url_base}&{params}'

result = requests.get(BASE_URI, params=params).json()

### Reading the output

The provided JSON output appears to contain data related to solar radiation, temperature, and wind conditions. Here's a breakdown of what each parameter likely represents:

- time: The timestamp of the data in the format YYYYMMDD:HHMM (year, month, day, hour, and minute). In this case, 20200102:1111 represents January 2, 2020, at 11:11.
- P: This typically represents Power (W/m²), specifically the potential photovoltaic power output. It likely indicates the estimated power generated by a solar panel under the given conditions.
- G(i): This is the global irradiance on an inclined plane (W/m²). It represents the amount of solar energy received per unit area on a surface inclined at a specific angle.
- H_sun: This is likely the solar height angle (degrees). It indicates the elevation angle of the sun above the horizon at the given time.
- T2m: This represents the air temperature at 2 meters above ground level (°C). It provides the ambient temperature near the Earth's surface.
- WS10m: This is the wind speed at 10 meters above ground level (m/s). It measures the velocity of the wind at that height.
- Int: This could represent solar radiation integral (kWh/m²) or another integrated measure of solar energy over a time interval. However, the value 0.0 suggests it might be indicating no accumulated energy during this specific moment or a placeholder for missing data.

In [5]:
result

{'inputs': {'location': {'latitude': 52.520008,
   'longitude': 13.404954,
   'elevation': 37.0},
  'meteo_data': {'radiation_db': 'PVGIS-SARAH3',
   'meteo_db': 'ERA5',
   'year_min': 2020,
   'year_max': 2021,
   'use_horizon': True,
   'horizon_db': None,
   'horizon_data': 'DEM-calculated'},
  'mounting_system': {'fixed': {'slope': {'value': 0, 'optimal': False},
    'azimuth': {'value': 0, 'optimal': False},
    'type': 'free-standing'}},
  'pv_module': {'technology': 'c-Si', 'peak_power': 86.4, 'system_loss': 0.0}},
 'outputs': {'hourly': [{'time': '20200101:0011',
    'P': 0.0,
    'G(i)': 0.0,
    'H_sun': 0.0,
    'T2m': 1.77,
    'WS10m': 2.41,
    'Int': 0.0},
   {'time': '20200101:0111',
    'P': 0.0,
    'G(i)': 0.0,
    'H_sun': 0.0,
    'T2m': 1.48,
    'WS10m': 2.48,
    'Int': 0.0},
   {'time': '20200101:0211',
    'P': 0.0,
    'G(i)': 0.0,
    'H_sun': 0.0,
    'T2m': 1.09,
    'WS10m': 2.41,
    'Int': 0.0},
   {'time': '20200101:0311',
    'P': 0.0,
    'G(i)': 0.0

## Using the result of the API